In [1]:
import itertools as it
from collections import defaultdict, Counter
from functools import reduce
from operator import add
import pandas as pd
import numpy as np

### Example: List out all possible matches

In [2]:
guys = list("ABC")
girls = list("FGH")

In [3]:
def reduce_path(path):
    """
    Reduces a path of matches to an unnested set of individuals.
    """
    return reduce(add, path)


def unique_couples(path):
    """
    Tests whether a path of matches has unique individuals.
    """
    count = Counter(reduce_path(path))
    return not any(c > 1 for c in count.values())


def generate_unique_paths(guys, girls):
    """
    Generate all unique paths (sets of couples).
    Returns a list of lists (paths) of tuples (couples).
    """
    # get all possible pairings
    product = list(it.product(guys, girls))
    
    # group pairings by guy
    dd = defaultdict(list)
    for guy, girl in product:
        dd[guy].append((guy, girl,))

    groups = list(dd.values())
    unique_paths = [list(path)
                    for path in it.product(*groups)
                    if unique_couples(path)]
    
    return unique_paths

    

In [4]:
paths = generate_unique_paths(guys, girls)
paths

[[('A', 'F'), ('B', 'G'), ('C', 'H')],
 [('A', 'F'), ('B', 'H'), ('C', 'G')],
 [('A', 'G'), ('B', 'F'), ('C', 'H')],
 [('A', 'G'), ('B', 'H'), ('C', 'F')],
 [('A', 'H'), ('B', 'F'), ('C', 'G')],
 [('A', 'H'), ('B', 'G'), ('C', 'F')]]

In [9]:
def gen_contestants(N):
    """
    Generate a unique set of girls and guys for N couples.
    Returns a dictionary of each set.
    """
    return dict(
        girls=[f'girl_{str(i).zfill(2)}' for i in range(1, N + 1)],
        guys=[f'guy_{str(i).zfill(2)}' for i in range(1, N + 1)]
    )

girls, guys = gen_contestants(10).values()

paths = generate_unique_paths(guys, girls)
print(f"Number of paths: {len(paths)}")

print("First 10 paths:")
for path in paths[:10]:
    print(path)

### OLD CODE ...

In [2]:
N = 3
guys = list('ABC')
girls = list('FGH')
perms = it.permutations(girls, N)

matches = {tuple(p): 1
           for perm in perms
           for p in zip(guys, perm)}
matches

{('A', 'F'): 1,
 ('B', 'G'): 1,
 ('C', 'H'): 1,
 ('B', 'H'): 1,
 ('C', 'G'): 1,
 ('A', 'G'): 1,
 ('B', 'F'): 1,
 ('C', 'F'): 1,
 ('A', 'H'): 1}

In [3]:
for (i, c), v in matches.items():
    print(i, c, v)

A F 1
B G 1
C H 1
B H 1
C G 1
A G 1
B F 1
C F 1
A H 1


In [5]:
N = 10
guys = range(N)
girls = range(N)
perms = it.permutations(girls, N)
    
paths = [list(zip(guys, perm)) for perm in perms]
n = len(paths)
print(f'{n:,.0f} paths')

3,628,800 paths


In [6]:
idx = pd.Index(range(10), name='Guys')
cols = pd.Index(range(10), name='Girls')

In [7]:
path = paths[0]
path

[(0, 0),
 (1, 1),
 (2, 2),
 (3, 3),
 (4, 4),
 (5, 5),
 (6, 6),
 (7, 7),
 (8, 8),
 (9, 9)]

In [8]:
def gen_matrix(males, females):
    z = np.zeros((len(males), len(females)),
                 dtype='int')
    z[males, females] = 1
    return z

perms = it.permutations(girls, N)
mats = np.concatenate([gen_matrix(guys, g) for g in perms])
mats.shape

In [16]:
n = mats.shape[0]
idx = pd.Index(list(range(N)) * int(n/N), dtype='int', name='Guys')
M = pd.DataFrame(mats, index=idx, columns=cols)
data = np.array([i for i in range(int(n/N)) for _ in range(N)],
                dtype='int')
M['Iteration'] = data
M = M.set_index('Iteration', append=True)
M.head(15)

,Girls,0,1,2,3,4,5,6,7,8,9
Guys,Iteration,,,,,,,,,,
0,0,1,0,0,0,0,0,0,0,0,0
1,0,0,1,0,0,0,0,0,0,0,0
2,0,0,0,1,0,0,0,0,0,0,0
3,0,0,0,0,1,0,0,0,0,0,0
4,0,0,0,0,0,1,0,0,0,0,0
5,0,0,0,0,0,0,1,0,0,0,0
6,0,0,0,0,0,0,0,1,0,0,0
7,0,0,0,0,0,0,0,0,1,0,0
8,0,0,0,0,0,0,0,0,0,1,0


#### Check for dupes

In [17]:
M.reset_index().duplicated().sum()

0

In [18]:
for path in paths[:5]:
    print(path)

[(0, 0), (1, 1), (2, 2), (3, 3), (4, 4), (5, 5), (6, 6), (7, 7), (8, 8), (9, 9)]
[(0, 0), (1, 1), (2, 2), (3, 3), (4, 4), (5, 5), (6, 6), (7, 7), (8, 9), (9, 8)]
[(0, 0), (1, 1), (2, 2), (3, 3), (4, 4), (5, 5), (6, 6), (7, 8), (8, 7), (9, 9)]
[(0, 0), (1, 1), (2, 2), (3, 3), (4, 4), (5, 5), (6, 6), (7, 8), (8, 9), (9, 7)]
[(0, 0), (1, 1), (2, 2), (3, 3), (4, 4), (5, 5), (6, 6), (7, 9), (8, 7), (9, 8)]


In [19]:
tots = M.groupby(level='Iteration').sum()
tots.describe()

Girls,0,1,2,3,4,5,6,7,8,9
count,3628800.0,3628800.0,3628800.0,3628800.0,3628800.0,3628800.0,3628800.0,3628800.0,3628800.0,3628800.0
mean,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
std,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
min,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
25%,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
50%,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
75%,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
max,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [20]:
tots.sum(1).value_counts()

10    3628800
dtype: int64

In [21]:
all = slice(None)
M.loc[(3, all), 4]

Guys  Iteration
3     0            0
      1            0
      2            0
      3            0
      4            0
                  ..
      3628795      0
      3628796      0
      3628797      0
      3628798      0
      3628799      0
Name: 4, Length: 3628800, dtype: int64

### Now that we have all possible matches, what is the best structure for working with them?
* A single matrix of all scenarios?
* A series of matrices?

### Need to be able to drop paths with invalid matches
* Could use brute force method and just search all paths for specific matches.  Is there a better way?

### How will you account for overall results?
* On the first round, all paths have equal probability and hence are equally weighted.
* On subsequent rounds, you will update the weights based on the results for that round.

In [22]:
import are_you_the_one as ayto

In [23]:
guys = [ayto.Guy(letter) for letter in list('ABCDE')]
girls = [ayto.Girl(letter) for letter in list('FGHIJ')]
tournament = ayto.Tournament(guys, girls)

In [24]:
tournament.grid.X

,F,G,H,I,J
A,0,0,0,0,0
B,0,0,0,0,0
C,0,0,0,0,0
D,0,0,0,0,0
E,0,0,0,0,0
